In [1]:
using BenchmarkTools
using Random

In [7]:
x = Vector{Int64}()
for y in 1:10
    push!(x, y)
end

In [8]:
x = [y for y in 1:10]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [10]:
x = Vector{Int64}()
@benchmark for y in 1:100000
    push!(x, y)
end

BenchmarkTools.Trial: 933 samples with 1 evaluation.
 Range (min … max):  2.522 ms …   1.706 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.537 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.812 ms ± 63.109 ms  ┊ GC (mean ± σ):  2.75% ± 4.46%

          ▃▅▆█▆▅▄                                             
  ▂▂▃▃▃▃▅▆█████████▅▅▃▄▃▄▂▃▃▃▃▃▃▃▂▁▃▂▂▂▁▃▃▃▃▃▄▄▃▃▃▂▂▂▂▂▁▂▁▁▂ ▃
  2.52 ms        Histogram: frequency by time        7.09 ms <

 Memory estimate: 1.52 MiB, allocs estimate: 99489.

In [11]:
@benchmark x = [y for y in 1:100000]

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   54.730 μs …  1.526 ms  ┊ GC (min … max): 0.00% … 75.93%
 Time  (median):      79.945 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   115.177 μs ± 92.190 μs  ┊ GC (mean ± σ):  0.65% ±  2.19%

  ▇█▇▇▆▆▅▅▄▄▃▃▃▂▂▁▂▁▁               ▂▃ ▃▂▁ ▂▁▁  ▁▁             ▂
  ██████████████████████▇▇▅▅▄▅▅▃▃▂▂▄██▇██████████████▇▇▇▇▇▅▇▇▇ █
  54.7 μs       Histogram: log(frequency) by time       416 μs <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

In [12]:
x = [5, 1, 5, 6, 7, 1, 4, 5]

8-element Vector{Int64}:
 5
 1
 5
 6
 7
 1
 4
 5

In [3]:
function jostle(x::Vector{Int64}, count::Integer = 2000000)
    jostles = Vector{Pair{Int64, Vector{Int64}}}()
    for shift in 1:count
        switch1, switch2 = rand(1:length(x)), rand(1:length(x))
        if switch1 == switch2
            switch2 = rand(1:length(x))
        end
        n1, n2 = x[switch1], x[switch2]
        x[switch1] = n2
        x[switch2] = n1
        push!(jostles, shift => [n1, n2])
    end
    jostles
end

jostle (generic function with 2 methods)

In [4]:
function compjostle(x::Vector{Int64}, count::Integer = 2000000)
    [begin
        switch1, switch2 = rand(1:length(x)), rand(1:length(x))
        if switch1 == switch2
            switch2 = rand(1:length(x))
        end
        n1, n2 = x[switch1], x[switch2]
        x[switch1] = n2
        x[switch2] = n1
        shift => [n1, n2]
            end for shift in 1:count]
end

compjostle (generic function with 2 methods)

In [5]:
@benchmark y = compjostle(x)

BenchmarkTools.Trial: 24 samples with 1 evaluation.
 Range (min … max):  159.673 ms … 417.888 ms  ┊ GC (min … max):  0.00% … 59.73%
 Time  (median):     184.987 ms               ┊ GC (median):    23.05%
 Time  (mean ± σ):   210.191 ms ±  62.849 ms  ┊ GC (mean ± σ):  28.39% ± 14.33%

     █ ▅   ▂                                                     
  ▅▁▁███▅▅▅█▁▁▁▅▅▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  160 ms           Histogram: frequency by time          418 ms <

 Memory estimate: 183.11 MiB, allocs estimate: 2000002.

In [6]:
@benchmark y = jostle(x)

BenchmarkTools.Trial: 20 samples with 1 evaluation.
 Range (min … max):  190.697 ms … 454.834 ms  ┊ GC (min … max):  0.00% … 48.03%
 Time  (median):     228.631 ms               ┊ GC (median):    22.54%
 Time  (mean ± σ):   251.905 ms ±  69.560 ms  ┊ GC (mean ± σ):  28.04% ± 15.03%

     █ █      ▃  ▃                                               
  ▇▁▇█▇█▇▁▁▁▁▇█▁▇█▁▁▁▁▇▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▇ ▁
  191 ms           Histogram: frequency by time          455 ms <

 Memory estimate: 190.40 MiB, allocs estimate: 2000015.